# Pràctica 3 - Llibreria spaCy
# ANTONIO JOSÉ LÓPEZ MARTÍNEZ
## ALEJANDRO MADRID GALARZA
En aquesta pràctica farem servir la llibreria spaCy per preprocessar text. Veurem els components següents: 
- Anàlisi morfològica.
- Anàlisi de dependències.
- Detecció d'entitats.
- Cerca de patrons amb la classe `Matcher`.
- Extensió d'atributs.
- Components personalitzats.

Carrega el contingut del fitxer `noticia.txt` i processa el text amb la llibreria spaCy en un objecte `doc`.

In [82]:
import spacy
from spacy import displacy

nlp = spacy.load("es_core_news_sm")

In [83]:
with open("./P3_materials/noticia_new.txt", "r", encoding="UTF-8") as f:
    noticia=f.read()
    
doc = nlp(noticia)
displacy.render(doc, style="ent")

Analitzem les característiques generals del text processat pel model a ES de `spaCy`. Calcula:  
- nombre de tokens en total
- nombre de lexemes diferents en total
- nombre de lexemes diferents traient espais i signes de puntuació
- nombre de lexemes diferents en minúscules traient espais i signes puntuació
- nombre de lemes diferents en minúscules traient espais i signes puntuació



In [84]:
# Tokens
print("El numero de tokens es: {}".format(len(doc)))
# Lexemas diferentes
lexemas_unicos = set(token.text for token in doc)
print("El numero de lexemas es: {}".format(len(lexemas_unicos)))
# Lezemas diferentes sin espacios ni signos de puntuación
lexemas_3 = set(token.text for token in doc if not token.is_space and not token.is_punct)
print("El numero de lexemas sin espacios ni signos de puntuación es: {}".format(len(lexemas_3)))
# Lexemas diferentes en minúsculas sin espacios ni signos de puntuación
lexemas_4 = set(token.text.lower() for token in doc if not token.is_space and not token.is_punct)
print("El numero de lexemas en minúsculas sin espacios ni signos de puntuación es: {}".format(len(lexemas_4)))
# Lemas diferentes en minúsculas sin espacios ni signos de puntuación
lemas_5 = set(token.lemma_.lower() for token in doc if not token.is_space and not token.is_punct)
print("El numero de lemas en minúsculas sin espacios ni signos de puntuación es: {}".format(len(lemas_5)))

El numero de tokens es: 543
El numero de lexemas es: 237
El numero de lexemas sin espacios ni signos de puntuación es: 231
El numero de lexemas en minúsculas sin espacios ni signos de puntuación es: 216
El numero de lemas en minúsculas sin espacios ni signos de puntuación es: 194


Calcula el nombre d'oracions del text.

In [85]:
print("El número de oraciones es : {}".format(len([s for s in doc.sents])))

El número de oraciones es : 18


### Cerca d'entitats
Enumera els tipus d'entitats al text amb el nombre d'aparicions de cada tipus.

In [86]:
# Importamos Counter de la librería collections
from collections import Counter
# Sacamos las entidades del text
apariciones_entidades = Counter(entidad.label_ for entidad in doc.ents)
# Printeamos las entidades
for tipo_entidad, apariciones in apariciones_entidades.items():
    print(f"Tipo de entidad: {tipo_entidad}, Apariciones: {apariciones}")

Tipo de entidad: ORG, Apariciones: 12
Tipo de entidad: MISC, Apariciones: 18
Tipo de entidad: PER, Apariciones: 3
Tipo de entidad: LOC, Apariciones: 38


## Anàlisi morfològica
Quins són els diferents tipus de POS (part-of-speech) que conté el text?

In [104]:
# Inicializamos el pos
tipos_pos = set()
for token in doc:
    tipos_pos.add(token.pos_)

# Print de los POS
print("Diferentes tipos de POS en el texto:")
for tipo_pos in tipos_pos:
    print(tipo_pos)
print(len(tipos_pos))

Diferentes tipos de POS en el texto:
VERB
DET
ADP
PROPN
AUX
ADJ
SPACE
NOUN
8


Indica la quantitat de substantius que hi ha al document (incloent-hi noms propis i pronoms).

In [88]:
cantidad_substantivos = 0

# Etiquetamos cada token y contamos los sustantivos
for token in doc:
    # Si token es sustantivo
    if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "PRON":
        cantidad_substantivos += 1

print("Cantidad de sustantivos en el documento:", cantidad_substantivos)


Cantidad de sustantivos en el documento: 177


Indica la quantitat d'adjectius en *femení* que hi ha al document.

In [105]:
cantidad_af = 0

# Etiquetamos cada token y contamos los adjetivos en femenino
for token in doc:
    #print(token.morph.get("Gender") == ['Fem'], token.pos_)
    # Si el token es un adjetivo en femenino, incrementamos el contador
    if token.pos_ == "ADJ" and token.morph.get("Gender") == ['Masc']:
        cantidad_af += 1

# Imprimimos la cantidad de adjetivos en femenino encontrados
print("Cantidad de adjetivos en femenino en el documento:", cantidad_af)

Cantidad de adjetivos en femenino en el documento: 1


## Cerca de patrons amb la classe `Matcher`

Usant la classe `Matcher` de la llibreria `spaCy` es poden cercar seqüències de paraules amb unes condicions molt particulars.

Fes servir la classe `Matcher` per buscar les seqüències de substantius en femení seguits d'un adjectiu.

Nota: consulta l'ajuda de la classe `Matcher` per veure com es defineixen els atributs morfològics a la cerca: https://spacy.io/usage/rule-based-matching#matcher

In [90]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
# Patron
patron = [{"POS": "NOUN", "MORPH": {"IS_SUPERSET": ["Gender=Fem"]}}, {"POS": "ADJ"}]
matcher.add("sustantivo_af", [patron] )
matches = matcher(doc)

# Print
for match_id, inicio, fin in matches:
    span = doc[inicio:fin]
    print("Se encontró el patrón:", span.text)

Se encontró el patrón: declaración conjunta
Se encontró el patrón: candidatura india
Se encontró el patrón: proliferación nuclear
Se encontró el patrón: armas nucleares
Se encontró el patrón: candidatura japonesa


## Anàlisi de dependències
Una altra manera de fer aquesta cerca consisteix a recórrer tots els substantius del document i veure per als seus tokens 'fill' quins són adjectius.  
Repeteix la cerca amb aquest mètode (ara han d'aparèixer adjectius anteriors i posteriors a cada substantiu).

In [91]:
for token in doc:
    
    if token.pos_ == "NOUN":
        print("Sustantivo:", token.text)
        adjectivos = []
        for child in token.children:
            if child.pos_ == "ADJ":
                adjectivos.append(child.text)
        if adjectivos:
            print("Adjetivos:", adjectivos)

Sustantivo: propuestas
Sustantivo: reforma
Sustantivo: conferencia
Sustantivo: presente
Sustantivo: historiador
Adjetivos: ['británico']
Sustantivo: mundo
Sustantivo: estructura
Adjetivos: ['presente']
Sustantivo: consenso
Sustantivo: alcance
Sustantivo: conversaciones
Sustantivo: número
Adjetivos: ['permanente']
Sustantivo: miembros
Sustantivo: países
Sustantivo: demandas
Adjetivos: ['fuertes']
Sustantivo: asientos
Adjetivos: ['permanentes']
Sustantivo: poderes
Adjetivos: ['principales', 'derrotados']
Sustantivo: segundos
Adjetivos: ['financiadores']
Sustantivo: terceros
Sustantivo: contribuidores
Adjetivos: ['mayores']
Sustantivo: tropas
Sustantivo: misiones
Sustantivo: paz
Sustantivo: propuesta
Sustantivo: oposición
Sustantivo: grupo
Sustantivo: países
Adjetivos: ['llamado']
Sustantivo: secretario
Adjetivos: ['anterior']
Sustantivo: equipo
Sustantivo: asesores
Sustantivo: recomendaciones
Sustantivo: año
Sustantivo: medida
Sustantivo: propuesta
Sustantivo: número
Adjetivos: ['permane

# Extensió d'atributs

`spaCy` permet afegir atributs personalitzats per als objectes `Doc`,
`Token` i `Span` per guardar dades específiques a les teues necessitats.

### Afegint atributs personalitzats

- Afegeix metadades personalitzades a documents, tokens i spans.
- Accessible a través de la propietat `._`

```python
doc._.title = "Mi documento"
token._.is_color = True
span._.has_color = False
```

- Es registren als `Doc`, `Token` o `Span` globals usant el mètode `set_extension`,

```python
# Importeu les classes globals
from spacy.tokens import Doc, Token, Span

# Afegeix extensions per al Doc, Token i Span
Doc.set_extension("title", default=None)
Token.set_extension("is_color", default=False)
Span.set_extension("has_color", default=False)
```

Els atributs personalitzats us permeten afegir metadades als docs, tokens i spans. Les dades poden ser afegides una vegada, o calculades dinàmicament.

Els atributs personalitzats estan disponibles a través de la propietat `._` (punt i guió baix). Aquesta notació fa que siga clar que han estat agregats per l'usuari i no estan integrats a spaCy com si ho és, per exemple, `token.text`.

Els atributs han de ser registrats a les classes `Doc`, `Token` i `Span` globals que pots importar des de `spacy.tokens`. Ja vas treballar amb elles als capítols anteriors. Per registrar un atribut personalitzat als `Doc`, `Token` i `Span`, pots utilitzar el mètode `set_extension`.

El primer argument és el nom de l'atribut. Els arguments *keyword* us permeten definir com ha de ser calculat el valor. En aquest cas, té un valor per defecte i pot ser sobreescrit.

Hi ha tres tipus d‟extensió: extensió d'atributs, extensió de propietats i extensió de mètodes.

### Extensió d'atributs

- Afegir un valor per defecte que pot ser sobreescrit.

```python
from spacy.tokens import Token

# Afegeix una extensió al Token amb un valor per defecte.
Token.set_extension("is_color", default=False)

doc = nlp("El cielo es azul.")

# Sobreescriu el valor de l'extensió d'atribut.
doc[3]._.is_color = True
```

Les extensions d'atribut afegeixen un valor per defecte que es pot sobreescriure.

Per exemple, un atribut personalitzat del token, anomenat `is_color`, que té per defecte el valor `False`.

En tokens individuals el seu valor pot ser canviat quan se sobreescriu - en aquest cas, `True` per al token "azul".

### Extensió de propietats

- Defineix una funció `getter` i una funció `setter` opcional.
- La funció `getter` només és cridada quan _es consulta_ el valor de l'atribut.

```python
from spacy.tokens import Token

# Defineix la funció "getter".
def get_is_color(token):
    colors = ["rojo", "amarillo", "azul"]
    return token.text in colors

# Afegeix una extensió al Token amb "getter".
Token.set_extension("is_color", getter=get_is_color)

doc = nlp("El cielo es azul.")
print(doc[3]._.is_color, "-", doc[3].text)
```

```out
True - azul
```

Les extensions de propietats funcionen com les propietats de Python: poden definir una funció <abbr title="En español: obtenedor. Una función que obtiene y devuelve un valor y que Python ejecuta automáticamente cuando se accede a un atributo especial de un objeto.">getter</abbr> i una funció <abbr title="En español: establecedor. Una función que de alguna forma establece un valor y que Python ejecuta automáticamente cuando se asigna un valor a un atributo especial de un objeto.">setter</abbr> opcional.

La funció `getter` només és cridada quan consultes l'atribut. Això permet calcular el valor dinàmicament, i fins i tot pot tenir en compte altres atributs personalitzats.

Les funcions `getter` prenen un argument: l'objecte, en aquest cas el token. En aquest exemple, la funció retorna si el text d'un token és a la nostra llista de colors.

Podem proveir la funció mitjançant l'argument keyword `getter` quan enregistrem l'extensió.

El token "azul" ara retorna `True` per `._.is_color`.

- Les extensions de `Span` gairebé sempre haurien d'usar un `getter`.

```python
from spacy.tokens import Span

# Defineix la funció "getter".
def get_has_color(span):
    colors = ["rojo", "amarillo", "azul"]
    return any(token.text in colors for token in span)

# Afegeix una extensió a `Span` amb `getter`.
Span.set_extension("has_color", getter=get_has_color)

doc = nlp("El cielo es azul.")
print(doc[1:4]._.has_color, "-", doc[1:4].text)
print(doc[0:2]._.has_color, "-", doc[0:2].text)
```

```out
True - cielo es azul
False - El cielo
```

Si voleu afegir extensions d'atributs a un `span`, gairebé sempre heu d'utilitzar una extensió de propietats amb un `getter`. D'altra manera, hauríeu d'actualitzar a mà _cadascun dels spans possibles_ per afegir tots els valors.

En aquest exemple, la funció `get_has_color` pren el `span` i torna si el text d'algun dels tokens és a la llista de colors.

Després d'haver processat el doc, podem revisar els diferents slices del doc i la propietat personalitzada `._.has_color` ens tornarà un resultat sobre si l'span conté un token de color o no.

### Extensió de mètodes

- Assigna una **funció** que passa a estar disponible com a mètode d'un objecte.
- Vos permet passar **arguments** a la funció d'extensió.

```python
from spacy.tokens import Doc

# Defineix un mètode amb arguments.
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# Afegeix una extensió al Doc amb el mètode.
Doc.set_extension("has_token", method=has_token)

doc = nlp("El cielo es azul.")
print(doc._.has_token("azul"), "- azul")
print(doc._.has_token("nube"), "- nube")
```

```out
True - azul
False - nube
```

L'extensió de mètodes fa que l'extensió de l'atribut siga un mètode que es pot cridar.

Pots passar-li un o més arguments i calcular els valors de l'atribut de manera dinàmica – per exemple, basats en cert argument o configuració.

En aquest exemple, la funció del mètode revisa si el doc conté un token amb un text donat. El primer argument del mètode és sempre lobjecte en si - en aquest cas, el doc. Es passa automàticament quan es truca al mètode.

Tots els altres arguments de la funció seran arguments a l'extensió del mètode. En aquest cas, `token_text`.

Ací el mètode personalitzat, `._.has_token`, torna `True` per a la paraula "azul" i `False` per a la paraula "nube".

## Afegint extensions d'atributs

Practicarem afegint algunes extensions d'atributs.

### Atributs de Token

- Completa la funció `is_country` que torna `True` si el token és a la llista de països.
- Usa `Token.set_extension` per registrar `"is_country"` (funció getter `get_is_country`).
- Imprimeix-ho en pantalla per a tots els tokens del text d'exemple.
from spacy.tokens import Token


In [92]:
from spacy.tokens import Token

# Define la función getter
def get_is_country(token):
    paises = ["España", "Alemania", "Francia", "Italia"]
    return token.text in paises

# Registra l'extensió de l'atribut del Token, "is_country", amb el getter "get_is_country".
Token.set_extension("is_country", getter=get_is_country, force=True)

# Procesa el text.
doc = nlp("Vivo en España.")

# Mostra en pantalla el text del token i el seu atribut "is_country" per a tots els tokens.
print([(token.text, token._.is_country) for token in doc])


[('Vivo', False), ('en', False), ('España', True), ('.', False)]


### Atributs de document

- Completa la funció `get_has_number`.
- Utilitza `Doc.set_extension` per a registrar `"has_number"` (getter `get_has_number`) i mostrar el seu valor per pantalla.
from spacy.tokens import Doc

In [106]:
from spacy.tokens import Doc

# Defineix la función getter
def get_has_number(doc):
    # Torna si algun dels tokens en el doc dona "True" per a token.like_num
    return any(token.like_num for token in doc)


# Registra l'extensió de propietat del Doc, "has_number", amb el getter "get_has_number".
Doc.set_extension("has_number", getter=get_has_number, force=True)


# Processa el text i revisa l'atribut personalitzat "has_number".
doc = nlp("Ese gato tiene tres piernas.")
print("has_number:", doc._.has_number)

has_number: True


### Atributs de Span

- Utilitza `Span.set_extension` per a registrar `"to_html"` (mètod `to_html`).
- Utilitza'l sobre `doc[0:2]` amb el tag `"strong"`.

In [95]:
from spacy.tokens import Span

# Defineix el mètod
def to_html(span, tag):
    # Embolica el text de l'span en un HTML tag i torna-ho.
    return f"<{tag}>{span.text}</{tag}>"


# Registra l'extensió de propietat de l'Span, "to_html", amb el mètode "to_html".
Span.set_extension("to_html", method=to_html, force = True)

# Processa el text i utilitza el mètod "to_html" en l'span amb el nom de tag "strong".
doc = nlp("Hola mundo, esto es una frase.")
span = doc[0:2]
print(span._.to_html("strong"))

<strong>Hola mundo</strong>


## Entitats i extensions

En aquest exercici combinaràs l'extensió d'atributs personalitzats amb les prediccions del model i crearàs un `getter` d'atribut que retorna una URL de cerca de Wikipedia si l'span és una persona, organització o lloc.

- Completa el getter `get_wikipedia_url` perquè només torne la URL si el text de l'span existeix literalment com a pàgina de la Wikipedia. Utilitza la documentació de l'API de la Wikipedia per completar la funció: https://api.wikimedia.org/wiki/Searching_for_Wikipedia_articles_using_Python
- Afegeix l'extensió de `Span`, `"wikipedia_url"`, usant el getter `get_wikipedia_url`.
- Itera sobre les entitats al `doc` de la notícia carregada inicialment i torna els seus URLs de Wikipedia.

Busquem les entitats a la Wikipedia per extreure el seu URL.

In [98]:
import requests
import json

number_of_results = 1
headers = {
  # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
  'User-Agent': 'GCD (etse.uv.es)'
}

url = 'https://api.wikimedia.org/core/v1/wikipedia/es/search/page'

def get_wikipedia_url(span):
    """
    Retorna la URL d'una entrada en la Wikipedia si existeix literalment.
    Args:
        objecte Span el text del qual es busca com a entrada a la Wikipedia.
    Returns:
        URL (string): enllaç a la wikipedia al text, o None si no existeix.
    """
    parameters = {'q': span.text, 'limit': number_of_results}
    response = requests.get(url, headers=headers, params=parameters)
    response = json.loads(response.text)['pages'][0]
    if response['title'] == span.text:
        return "https://es.wikipedia.org/wiki/" + response["key"]
    else:
        return None


# Afegeix l'extensió de l'Span, wikipedia_url, usant el getter get_wikipedia_url.
Span.set_extension("wikipedia_url", getter=get_wikipedia_url, force = True)

doc = nlp(
    "Antes de finalizar 1976, el interés de David Bowie en la "
    "floreciente escena musical alemana, le llevó a mudarse a "
    "Alemania para revitalizar su carrera."
)
for ent in doc.ents:
    # Imprimeix en pantalla el text i l'URL de la Viquipèdia de l'entitat.
    print(ent.text, ent._.wikipedia_url)

David Bowie https://es.wikipedia.org/wiki/David_Bowie
Alemania https://es.wikipedia.org/wiki/Alemania


# Components personalitzats del pipeline

`spaCy` permet crear components personalitzats per al pipeline.

Els components personalitzats del pipeline et permeten afegir la teua pròpia funció al pipeline de spaCy que s'executa quan crides a l'objecte `nlp` sobre un text - per exemple, per modificar el doc i afegir-hi més dades.

Després que el text és convertit en tokens i un objecte `Doc` ha estat creat, els components del pipeline s'apliquen en ordre. spaCy ofereix suport per a un rang de components inclosos, però també vos permet definir els vostres.

Els components personalitzats s'executen automàticament quan crides a l'objecte `nlp` sobre un text.

Són especialment útils per afegir les teues pròpies metadades als documents i als tokens.

També pots fer-los servir per actualitzar els atributs inclosos, com els spans de les entitats nomenades.

### Anatomia d'un component

- Funcions que prenen un `doc`, ho modifiquen i ho tornen.
- Poden ser afegits usant el mètode `nlp.add_pipe`.

```python
def custom_component(doc):
    # Fes alguna cosa amb del doc ací.
    return doc

nlp.add_pipe(custom_component)
```

Fonamentalment, un component del pipeline és una funció o un <abbr title="Algo que puede ser llamado o ejecutado, así como una función o una clase.">callable</abbr> que pren a un doc, el modifica i el torna perquè puga ser processat pel proper component al pipeline.

Els components poden ser afegits al pipeline usant el mètode `nlp.add_pipe`. Aquest pren almenys un argument: la funció del component.


| Argument | Descripció           | Exemple                                   |
| -------- | -------------------- | ----------------------------------------- |
| `last`   | Si és `True`, el posa d'últim  | `nlp.add_pipe(component, last=True)`      |
| `first`  | Si és `True`, el posa de primer | `nlp.add_pipe(component, first=True)`     |
| `before` | L'afegeix abans del component | `nlp.add_pipe(component, before="ner")`   |
| `after`  | L'afegeix després del component  | `nlp.add_pipe(component, after="tagger")` |

Per especificar _on_ afegir el component al pipeline, pots fer servir un dels següents arguments keyword:

Si feu que el valor de `last` siga `True`, s'afegirà el component a l'últim lloc del pipeline. Aquest és el comportament per defecte.

Si fas que el valor de `first` siga `True`, s'afegirà al primer lloc del pipeline just després del tokenizer.

Els arguments `before` i `after` vos permeten definir el nom d'un component existent al qual podeu afegir el nou component abans o després. Per exemple, `before="ner"` afegirà el component nou abans del named entity recognizer.

No obstant això, l'altre component al qual s'afegirà un component nou abans o després ha d'existir. Si no, spaCy llançarà un error.

### Exemple: un component simple

```python
# Crea l'objecte nlp.
nlp = spacy.load("es_core_news_sm")

# Defineix un component personalitzat.
def custom_component(doc):
    # Mostra la longitud del doc en pantalla.
    print("longitud del Doc:", len(doc))
    # Torna l'objecte doc.
    return doc

# Afegeix el component al primer lloc del pipeline.
nlp.add_pipe(custom_component, first=True)

# Imprimeix els noms dels components del pipeline.
print("Pipeline:", nlp.pipe_names)
```

```out
Pipeline: ['custom_component', 'tagger', 'parser', 'ner']
```

```python
# Processa un text.
doc = nlp("¡Hola Mundo!")
```

```out
longitud del Doc: 4
```

Aquí tenim un exemple de component simple del pipeline.

Comencem amb el model petit d'espanyol.

Després definim el component - una funció que pren un objecte `Doc` i després el torna.

Fem alguna cosa simple i imprimim a la pantalla la longitud del document que passa pel pipeline.

No oblides tornar el doc perquè puga ser processat pel proper component al pipeline! El doc creat pel tokenizer passa per tots els components, així que és important que tots tornen el doc modificat.

Ara hi podem afegir el component al pipeline. Ho afegirem al primer lloc, just després del tokenizer. Ho fem definint `first=True`.

Quan imprimim a la pantalla els noms dels components el nou apareix al principi. Això vol dir que serà aplicat quan processem un doc.

Ara, quan processem un text usant l'objecte `nlp`, el component personalitzat serà aplicat al doc i la longitud del document serà impresa a la pantalla.

### Components simples

Completa l'exemple per mostrar un component personalitzat que imprimeix la longitud d'un document.

- Completa la funció del component amb la longitud del `doc`.
- Afegeix el `length_component` al pipeline existen com el **primer** component.
- Prova el nou pipeline i processa qualsevol text amb l'objecte `nlp` - per exemple, "Això és una frase."

In [99]:
from spacy.language import Language

# Defineix el component personalitzat.
@Language.component('length_component')
def length_component(doc):
    # Obtin la longitud del doc.
    doc_length = len(doc)
    print(f"Este documento tiene {doc_length} tokens.")
    # Torna el doc.
    return doc


# Carga el model petit d'espanyol.
nlp = spacy.load("es_core_news_sm")

# Afegeix el component en el primer lloc del pipeline i mostra els noms dels pipes en pantalla.
nlp.add_pipe('length_component', first=True)
print(nlp.pipe_names)

# Processa un text.
doc = nlp("¡Hola Mundo!")

['length_component', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Este documento tiene 4 tokens.


## Components amb extensions

L'extensió d'atributs és especialment poderosa si és combinada amb els components personalitzats del pipeline. En aquest exercici escriuràs un component del pipeline que troba noms de països i una extensió d'atribut personalitzada que torna la ciutat capital del país si està disponible.

Un patró de frases amb tots els països està disponible com a variable `matcher`. Un diccionari de països relacionats amb les seves ciutats cabdals està disponible com la variable `CAPITALS`.

- Completa el `countries_component` i crea un `Span` amb el label `"LOC"` per a tots els resultats.
- Afegeix el component al pipeline.
- Registra l'extensió de l'atribut de l'Span, `"capital"`, amb el getter `get_capital`.
- Processa el text i mostra per pantalla el text de l'entitat, el label i la capital de l'entidad per a cada span que conté una entitat en `doc.ents`.

In [101]:
import json
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.language import Language

with open("P3_materials/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

with open("P3_materials/capitals.json", encoding="utf8") as f:
    CAPITALS = json.loads(f.read())

nlp = spacy.load("es_core_news_sm")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("COUNTRY", None, *list(nlp.pipe(COUNTRIES)))

@Language.component('countries_component')
def countries_component(doc):
    # Crea un Span d'entitats amb el label "LOC" per a tots los resultats.
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="LOC") for match_id, start, end in matches]
    return doc


# Afegeix el component al pipeline.
nlp.add_pipe('countries_component')
print(nlp.pipe_names)

# El getter que busca el text de l'span en un diccionari de ciudats capitals de països.
get_capital = lambda span: CAPITALS.get(span.text)

# Registra l'extensió d'atributo de l'Span, "capital", amb el getter "get_capital".
Span.set_extension("capital", getter=get_capital, force=True)

# Processa el text i mostra per pantalla el text de l'entidad, el label i els atributs "capital".
doc = nlp(
    "La República Checa podría ayudar a la República Eslovaca "
    "a proteger su espacio aéreo"
)
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'countries_component']
[('República Checa', 'LOC', 'Prague'), ('República Eslovaca', 'LOC', 'Bratislava')]


En registrar el nou component `countries_component` es perd la detecció de la resta d'entitats del Doc. Torna a carregar el pipeline però aquesta vegada registra el nou component *ABANS* del `ner` perquè això no passe. Processa el text per veure quines entitats detecta.

In [102]:
import json
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.language import Language

with open("P3_materials/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

with open("P3_materials/capitals.json", encoding="utf8") as f:
    CAPITALS = json.loads(f.read())

nlp = spacy.load("es_core_news_sm")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("COUNTRY", None, *list(nlp.pipe(COUNTRIES)))

matcher.add("COUNTRY", None, *list(nlp.pipe(COUNTRIES)))

@Language.component('countries_component')
def countries_component(doc):
    # Crea un Span d'entitats amb el label "LOC" per a tots los resultats.
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="LOC") for match_id, start, end in matches]
    return doc


nlp = spacy.load("es_core_news_sm")
# Afegeix el component al pipeline.

nlp.add_pipe('countries_component', before='ner')
print(nlp.pipe_names)
doc = nlp(
    "La República Checa podría ayudar a la República Eslovaca \
    a proteger su espacio aéreo por la mediación de Joe Biden"
)
print([(ent.text, ent.label_) for ent in doc.ents])

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'countries_component', 'ner']
[('República Checa', 'LOC'), ('República Eslovaca', 'LOC'), ('Joe Biden', 'PER')]
